In [11]:
# -*- coding: utf-8 -*-
import scipy.io as scio
#import pandas as pd
import numpy as np
import random
import operator
import copy
import os
import math
#import matplotlib.pyplot as plt
#import matlab.engine
import time
import multiprocessing as mp
import warnings

In [12]:
# loading instance data
def load_data(file_path):
    
    matrix_data = scio.loadmat(file_path)
    
    google_duration = matrix_data['GoogleDurations']
    great_distance = matrix_data['GreatCircleDistances']
    XY = matrix_data['MyLocations']
    
    num_node = len(google_duration)
    location = np.ones((num_node,num_node),dtype=float)
    
    for i in range(num_node):
        google_duration[i][i] = 0
        great_distance[i][i] = 0
        
    for i in range(num_node):
        location[i,0] = XY[i][0][0,0]
        location[i,1] = XY[i][0][0,1]
    
    return google_duration, great_distance, location

In [13]:
def element_inquiry(greatcir_dis,max_node):
    
    n_node = len(greatcir_dis)
    logical_matrix = np.zeros((n_node,n_node),dtype=int)
    
    logical_matrix[0] = 1
    logical_matrix[:,0] = 1
    
    # according to the inquiry budget, specify the elements which are available from google distance matrix
    
    for k in range(n_node):
        for row_id in range(n_node):
            cur_row_data = greatcir_dis[row_id]
            cur_row_data[row_id] = 100000000000
            near_node_id = np.argsort(cur_row_data)[k]
            cur_row_data[row_id] = 0
            logical_matrix[row_id,near_node_id] = 1
            logical_matrix[near_node_id,row_id] = 1
            if sum(sum(logical_matrix)) >= max_node:
                return logical_matrix
        
    

In [16]:
def knn_matrix_completion(logi_mat,greatcir_dis,circuity_factor,num_adj,sf):
    
    n_node = len(greatcir_dis)
    knn_logi_mat = np.zeros((n_node,n_node),dtype=float)
    # specify the position of known nodes inquired from the google distance matrix
    for i in range(n_node):
        for j in range(n_node):
            if logi_mat[i][j] == 1:
                knn_logi_mat[i][j] = circuity_factor[i][j]
                
#     ave_circuity = 0
#     count = 0
#     for i in range(n_node):
#         for j in range(n_node):
#             if knn_logi_mat[i][j] != 0:
#                 ave_circuity = ave_circuity + knn_logi_mat[i][j]
#                 count = count + 1
                
#     ave_circuity = ave_circuity/count           
                
    # infer the elements which are unknown in the smart distance matrix
    for i in range(n_node):
        for j in range(n_node):
            if logi_mat[i][j] == 0 and i!=j:
                alter_link = dict()
                
                for ii in range(n_node):
                    for jj in range(n_node):
                        if logi_mat[ii][jj]:
                            # distance between origins
                            dist_org = greatcir_dis[i,ii]
                            # distance between destinations
                            dist_dest = greatcir_dis[jj,j]
                            # save the sum of two distances
                            alter_link[(ii,jj)] = dist_org + dist_dest
                # sort_dict's datatype is list
                sorted_dict = sorted(alter_link.items(), key=operator.itemgetter(1))
                
                logical_mean = 0
                if len(sorted_dict) >= num_adj:
                    
                    for item in range(num_adj):
                        close_org = sorted_dict[item][0][0]
                        close_dest = sorted_dict[item][0][1]
                        logical_mean = logical_mean + circuity_factor[close_org][close_dest]
                        #logical_mean = logical_mean + circuity_factor[close_org][close_dest]
                    logical_mean = logical_mean/num_adj
                    
#                 else:
#                     if len(sorted_dict) > 0:
                            
#                         for item in range(len(sorted_dict)):
#                             close_org = sorted_dict[item][0][0]
#                             close_dest = sorted_dict[item][0][1]
#                             logical_mean = logical_mean + circuity_factor[close_org][close_dest]
#                             #logical_mean = logical_mean + circuity_factor[close_org][close_dest]
#                         logical_mean = logical_mean/len(sorted_dict)
                if logical_mean > 0:
                    knn_logi_mat[i][j] = logical_mean*sf
    ave_circuity = 0
    count = 0
    for i in range(n_node):
        for j in range(n_node):
            if knn_logi_mat[i][j] != 0:
                ave_circuity = ave_circuity + knn_logi_mat[i][j]
                count = count + 1
    ave_circuity = ave_circuity/count 
    for i in range(n_node):
        for j in range(n_node):
            if i != j and knn_logi_mat[i][j] == 0:
                knn_logi_mat[i][j] = ave_circuity
    return knn_logi_mat*greatcir_dis/1000

In [17]:
def matrix_completion(file_name, k_nn, scale_factor):
    
    # parameter setting
    inquiry_budget = 0.2
    
    #k_nn = 4
        
    #scale_factor = 1.1
    
    path = r"C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData"
    
    file_path = os.path.join(path, file_name)
    print(file_path)
    
    # load the data 
    [google_duration, great_distance, location] = load_data(file_path)
    num_node = len(google_duration)
    # calculate the circuity factors
    CircuityFactor = google_duration/(great_distance/1000)
    for i in range(num_node):
        for j in range(num_node):
            if np.isnan(CircuityFactor[i][j]):
                CircuityFactor[i][j] = 0
    
    # MaxElements denotes the total maximun nodes we can inquiry.
    MaxElements = int(inquiry_budget*num_node*num_node);
                
    # get the matrix showing which elements can be available
    access_matrix = element_inquiry(great_distance, MaxElements)
    
    # use knn framework to infer the whole distance matrix
    smart_distance = knn_matrix_completion(access_matrix,great_distance,CircuityFactor,k_nn,scale_factor);
    
    whole_data = np.vstack((great_distance,smart_distance,google_duration,location))
    
    np.savetxt(file_name.split('.')[0]+"_"+str(k_nn)+"_"+str(scale_factor)+'.txt', whole_data)
    
    
#     print("smart:")
#     print(smart_distance)
#     print("google:")
#     print(google_duration)
#     print("circuity")
#     print(CircuityFactor)
    
#     for k in range(num_node):
#         smart_distance[k][k] = 1
#         google_duration[k][k] = 1
        
    
    
#     print('mape (google vs. smart)')
#     print(np.mean(np.abs((smart_distance-google_duration) / google_duration)) * 100)
    
#     return np.mean(np.abs((smart_distance-google_duration) / google_duration)) * 100
    return 1
    

In [18]:
# run on lab chpc
start = time.time()
path = r"C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData"

# # parameter setting
# inquiry_budget = 0.2
warnings.filterwarnings("ignore")
# Scale_factor = 1
total_result = []


for KNN in range(8,9):

    for Scale_factor in [1]:
        print(KNN,Scale_factor)
        aveMAPE = 0
        for file in os.listdir(path):
        # run mainfunction
            aveMAPE = aveMAPE + matrix_completion(file,KNN,Scale_factor)
    
#         aveMAPE = aveMAPE/10
#         print("avemape",aveMAPE);
#         total_result.append([KNN,Scale_factor,aveMAPE])

end = time.time()
running_time = end-start
print(running_time)

8 1
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData\101x101_instance_1.mat
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData\101x101_instance_10.mat
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData\101x101_instance_2.mat
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData\101x101_instance_3.mat
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData\101x101_instance_4.mat
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData\101x101_instance_5.mat
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-BASIC-KNN\101TestData\101x101_instance_6.mat
C:\WYH\research project\traveltime_model_basedon_saving\MC Research - travel time-p1-